In [ ]:
from cdcqr.data.glassnode import glassnode_data as gnd
from cdcqr.ct.utils import plot2
import pandas as pd
import cdcqr.analytics.utils as autil 
from importlib import reload
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
import sys
sys.path.append('/core/github/cryptoderiv-quant/')
from ct.alfafactory import runsignal
import numpy as np
import os
from cdcqr.common.utils import LOCAL_DATA_DIR

In [ ]:
gn = gnd.GlassnodeData()

### BTC

In [ ]:
fs =  ['price_usd_close','FuturesFundingRatePerpetual']
a = ['BTC','ETH']
#i = '10m'
df = gn.load_features(fs,a)
df.head(2)

In [ ]:
cols = list(df.columns)
cols

In [ ]:
df1 = df[df.index>='20210101']
display(df1.head(2))
df1.plot2(cols[0],cols[1:3])

In [ ]:
reload(autil)
feature_raw = '{}_futures_funding_rate_perpetual'.format(a)
feature_normalized = feature_raw
target = '{}_price_usd_close'.format(a)
_ = autil.ts_eda(df1, feature_raw, feature_normalized, target, save_fig=True)

## backtest

In [ ]:
df1.head(5)

In [ ]:
df1['signal'] = df1['BTC-FuturesFundingRatePerpetual'].rolling(200).apply(pctrank)

In [ ]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))
p=price.resample('1Min').last().ffill()
request = ['pf','backtest']

In [ ]:
signalid = 'long-funding_rate'
res=[]
price = p.copy()
counter0 = 0
short = False # long strategy
for qtl in [0.995, 0.99, 0.95, 0.9]:
    signal = df1['signal']>qtl
    signal = signal.reindex(p.index).ffill()
    for tp in [3,5,7]:
        for sl in [0.5, 1, 2]:
            counter0 = counter0 + 1
            print(f'{counter0} : tp:{tp} sl:{sl} id:{signalid}')
            for n1 in [1,10,100]: #
                for n2 in [101, 200, 1000]:
                    ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=None,n2=None,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
                    for k in ressig:
                        resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/65} # rescale to number of weeks
                        res.append(resd)
                            
pd.DataFrame(res).pv()

In [ ]:
pd.DataFrame(res).pv()